In [1]:
from omegaconf import OmegaConf
import os
import torch

import hydra
from omegaconf import DictConfig, OmegaConf
from models.together_model import ProteinVAELLMmodel, ProteinVAELLM_FrameDiff_first_model
from data import all_atom
# Pytorch lightning imports
from pytorch_lightning import LightningDataModule, LightningModule, Trainer
from pytorch_lightning.loggers.wandb import WandbLogger
from pytorch_lightning.trainer import Trainer
from pytorch_lightning.callbacks import ModelCheckpoint

from data.pdb_dataloader import PdbDataModule
from models.flow_module import FlowModule
from experiments import utils as eu
from data import utils as du
from analysis import metrics
import openfold.utils.rigid_utils as ru
import torch.nn.functional as F
import subprocess
from biotite.sequence.io import fasta
cfg = OmegaConf.load("configs/base.yaml")
_cfg = cfg
_data_cfg = cfg.data
_exp_cfg = cfg.experiment
_datamodule: LightningDataModule = PdbDataModule(_data_cfg)
_datamodule.setup(stage="fit")
torch.autograd.set_detect_anomaly(True)

/gpfs/gibbs/project/dijk/sh2748/conda_envs/fm/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import shutil
pdb_path = "/home/sh2748/inference_outputs/baseline/2024-04-30_15-54-45/epoch=11-step=23628/run_2024-05-01_23-06/length_100/sample_0/sample.pdb"
sc_output_dir = os.path.join("/home/sh2748/inference_outputs/baseline/2024-04-30_15-54-45/epoch=11-step=23628/run_2024-05-01_23-06/length_100/sample_0", 'self_consistency')
os.makedirs(sc_output_dir, exist_ok=True)
        

In [3]:
os.path.basename(pdb_path), os.path.join(sc_output_dir, os.path.basename(pdb_path))

('sample.pdb',
 '/home/sh2748/inference_outputs/baseline/2024-04-30_15-54-45/epoch=11-step=23628/run_2024-05-01_23-06/length_100/sample_0/self_consistency/sample.pdb')

In [4]:
shutil.copy(pdb_path, os.path.join(sc_output_dir, os.path.basename(pdb_path)))        

'/home/sh2748/inference_outputs/baseline/2024-04-30_15-54-45/epoch=11-step=23628/run_2024-05-01_23-06/length_100/sample_0/self_consistency/sample.pdb'

In [5]:
import esm
from typing import Optional
import numpy as np
import pandas as pd

_pmpnn_dir = "./ProteinMPNN/"
seq_per_sample = 8
gpu_id = None
_folding_model = esm.pretrained.esmfold_v1().eval().to("cuda")

[2024-05-07 14:04:46,452] [INFO] [real_accelerator.py:203:get_accelerator] Setting ds_accelerator to cuda (auto detect)
 [WARNING]  async_io requires the dev libaio .so object and headers but these were not found.
 [WARNING]  async_io: please install the libaio-devel package with yum
 [WARNING]  If libaio is already installed (perhaps from source), try setting the CFLAGS and LDFLAGS environment variables to where it can be found.
 [WARNING]  Please specify the CUTLASS repo directory as environment variable $CUTLASS_PATH


/usr/bin/ld.gold: error: cannot find -laio
/tmp/tmp.ZHabli45NE/tmpjr_l5e7_/test.o:test.c:function main: error: undefined reference to 'io_pgetevents'
collect2: error: ld returned 1 exit status


 [WARNING]  sparse_attn requires a torch version >= 1.5 and < 2.0 but detected 2.0
 [WARNING]  using untested triton version (2.0.0), only 1.0.0 is known to be compatible


In [6]:
def run_self_consistency(
        decoy_pdb_dir: str,
        reference_pdb_path: str,
        motif_mask: Optional[np.ndarray]=None):
    """Run self-consistency on design proteins against reference protein.
        
        Args:
            decoy_pdb_dir: directory where designed protein files are stored.
            reference_pdb_path: path to reference protein file
            motif_mask: Optional mask of which residues are the motif.

        Returns:
            Writes ProteinMPNN outputs to decoy_pdb_dir/seqs
            Writes ESMFold outputs to decoy_pdb_dir/esmf
            Writes results in decoy_pdb_dir/sc_results.csv
    """

    # Run PorteinMPNN
    output_path = os.path.join(decoy_pdb_dir, "parsed_pdbs.jsonl")
    process = subprocess.Popen([
        'python',
        f'{_pmpnn_dir}/helper_scripts/parse_multiple_chains.py',
        f'--input_path={decoy_pdb_dir}',
        f'--output_path={output_path}',
    ])
    _ = process.wait()

    num_tries = 0
    ret = -1
    pmpnn_args = [
        'python',
        f'{_pmpnn_dir}/protein_mpnn_run.py',
        '--out_folder',
        decoy_pdb_dir,
        '--jsonl_path',
        output_path,
        '--num_seq_per_target',
        str(seq_per_sample),
        '--sampling_temp',
        '0.1',
        '--seed',
        '38',
        '--batch_size',
        '1',
    ]
    if gpu_id is not None:
        pmpnn_args.append('--device')
        pmpnn_args.append(str(gpu_id))
    while ret < 0:
        try:
            process = subprocess.Popen(
                pmpnn_args,
                stdout=subprocess.DEVNULL,
                stderr=subprocess.STDOUT
            )
            ret = process.wait()
        except Exception as e:
            num_tries += 1
            print(f'Failed ProteinMPNN. Attempt {num_tries}/5')
            torch.cuda.empty_cache()
            if num_tries > 4:
                raise e
    print(f"Return value: {ret}")
    mpnn_fasta_path = os.path.join(
        decoy_pdb_dir,
        'seqs',
        os.path.basename(reference_pdb_path).replace('.pdb', '.fa')
    )
    print(mpnn_fasta_path)
    # Run ESMFold on each ProteinMPNN sequence and calculate metrics.
    mpnn_results = {
        'tm_score': [],
        'sample_path': [],
        'header': [],
        'sequence': [],
        'rmsd': [],
    }
    if motif_mask is not None:
        # Only calculate motif RMSD if mask is specified.
        mpnn_results['motif_rmsd'] = []
    esmf_dir = os.path.join(decoy_pdb_dir, 'esmf')
    os.makedirs(esmf_dir, exist_ok=True)
    
    fasta_seqs = fasta.FastaFile.read(mpnn_fasta_path)
    sample_feats = du.parse_pdb_feats('sample', reference_pdb_path)
    for i, (header, string) in enumerate(fasta_seqs.items()):

        # Run ESMFold
        esmf_sample_path = os.path.join(esmf_dir, f'sample_{i}.pdb')
        _ = run_folding(string, esmf_sample_path)
        esmf_feats = du.parse_pdb_feats('folded_sample', esmf_sample_path)
        sample_seq = du.aatype_to_seq(sample_feats['aatype'])

        # Calculate scTM of ESMFold outputs with reference protein
        _, tm_score = metrics.calc_tm_score(
            sample_feats['bb_positions'], esmf_feats['bb_positions'],
            sample_seq, sample_seq)
        rmsd = metrics.calc_aligned_rmsd(
            sample_feats['bb_positions'], esmf_feats['bb_positions'])
        if motif_mask is not None:
            sample_motif = sample_feats['bb_positions'][motif_mask]
            of_motif = esmf_feats['bb_positions'][motif_mask]
            motif_rmsd = metrics.calc_aligned_rmsd(
                sample_motif, of_motif)
            mpnn_results['motif_rmsd'].append(motif_rmsd)
        mpnn_results['rmsd'].append(rmsd)
        mpnn_results['tm_score'].append(tm_score)
        mpnn_results['sample_path'].append(esmf_sample_path)
        mpnn_results['header'].append(header)
        mpnn_results['sequence'].append(string)

        # Save results to CSV
    csv_path = os.path.join(decoy_pdb_dir, 'sc_results.csv')
    mpnn_results = pd.DataFrame(mpnn_results)
    mpnn_results.to_csv(csv_path)


def run_folding(sequence, save_path):
    """Run ESMFold on sequence."""
    with torch.no_grad():
        output = _folding_model.infer_pdb(sequence)
    with open(save_path, "w") as f:
        f.write(output)
    return output

In [7]:
sc_output_dir

'/home/sh2748/inference_outputs/baseline/2024-04-30_15-54-45/epoch=11-step=23628/run_2024-05-01_23-06/length_100/sample_0/self_consistency'

In [8]:
run_self_consistency(decoy_pdb_dir=sc_output_dir, reference_pdb_path=pdb_path)

Return value: 0
/home/sh2748/inference_outputs/baseline/2024-04-30_15-54-45/epoch=11-step=23628/run_2024-05-01_23-06/length_100/sample_0/self_consistency/seqs/sample.fa


In [9]:
import os
import pandas as pd
import numpy as np
import plotnine as gg

def read_samples(results_dir):
    all_csvs = []
    print(f'Reading samples from {results_dir}')
    for sample_length in os.listdir(results_dir):
        if '.' in sample_length:
            continue
        length_dir = os.path.join(results_dir, sample_length)
        length = int(sample_length.split('_')[1])
        for i,sample_name in enumerate(os.listdir(length_dir)):
            if '.' in sample_name:
                continue
            csv_path = os.path.join(length_dir, sample_name, 'self_consistency', 'sc_results.csv')
            if os.path.exists(csv_path):
                design_csv = pd.read_csv(csv_path, index_col=0)
                design_csv['length'] = length
                design_csv['sample_id'] = i
                all_csvs.append(design_csv)
    results_df = pd.concat(all_csvs)
    return results_df


def sc_filter(raw_df, metric):
    # Pick best self-consistency sample
    if metric == 'tm_score':
        df = raw_df.sort_values('tm_score', ascending=False)
        df['designable'] = df.tm_score.map(lambda x: x > 0.5)
    elif metric == 'rmsd':
        df = raw_df.sort_values('rmsd', ascending=True)
        df['designable'] = df.rmsd.map(lambda x: x < 2.0)
    else:
        raise ValueError(f'Unknown metric {metric}')
    df = df.groupby(['length', 'sample_id']).first().reset_index()
    percent_designable = df['designable'].mean()
    print(f'Percent designable: {percent_designable}')
    return df

In [12]:

samples_df = read_samples("/home/sh2748/inference_outputs/baseline/2024-04-30_15-54-45/epoch=11-step=23628/run_2024-05-01_23-06")
samples_df = samples_df[samples_df.sample_id < 8] # Ensure we only consider 8 sequences per backbone.

scrmsd_results = sc_filter(samples_df, 'rmsd')
sctm_results = sc_filter(samples_df, 'tm_score')

Reading samples from /home/sh2748/inference_outputs/baseline/2024-04-30_15-54-45/epoch=11-step=23628/run_2024-05-01_23-06
Percent designable: 0.0
Percent designable: 0.0
